Hahua Mykyta DA-22 Machine Learning Lab#2

First of all lets download and read data from google drive urls, i uploaded them on google drive

In [354]:
import os
import gdown
import numpy as np
import math
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import pandas as pd
import matplotlib.pyplot as plt

train_url = 'https://drive.google.com/uc?id=1wCKXZWpYEysIFn1F74R9pTOt6aFasKTy'
test_url = 'https://drive.google.com/uc?id=1gTuliILSkSw9JYSW_RhpxcnqKS3Q-PMM'

train = 'application_train.csv'
test = 'application_test.csv'

if not os.path.exists(train):
    print(f"{train} not found. Downloading...")
    gdown.download(train_url, train, quiet=False)
else:
    print(f"{train} already exists. Skipping download.")

if not os.path.exists(test):
    print(f"{test} not found. Downloading...")
    gdown.download(test_url, test, quiet=False)
else:
    print(f"{test} already exists. Skipping download.")

df_train = pd.read_csv(train)
df_test = pd.read_csv(test)

application_train.csv not found. Downloading...


Downloading...
From (original): https://drive.google.com/uc?id=1wCKXZWpYEysIFn1F74R9pTOt6aFasKTy
From (redirected): https://drive.google.com/uc?id=1wCKXZWpYEysIFn1F74R9pTOt6aFasKTy&confirm=t&uuid=4e28f51a-9c1f-4a58-85a0-24981542bee0
To: /Users/nikitagagua/PycharmProjects/machine learning/application_train.csv
 10%|█         | 16.8M/166M [00:02<00:19, 7.58MB/s]

KeyboardInterrupt: 

get info about table

In [ ]:
df_train.info()

get info about feature types

In [ ]:
df_train.dtypes

lets get sample of 15 rows 

In [ ]:
print(df_train.shape)
df_train.head()
df_train.sample(15)

check for missing values:

In [ ]:

df_train.isnull().sum()

here we are going to delete features where missing ratio of feature is greater than 10%

In [ ]:
missing_ratio = df_train.isnull().sum() / len(df_train) * 100
print(missing_ratio[missing_ratio > 0].sort_values(ascending=False))

threshold = 0.1
columns_to_drop = missing_ratio[missing_ratio > threshold * 100].index
df_train.drop(columns=columns_to_drop, axis=1, inplace=True)
df_test.drop(columns=columns_to_drop, axis=1, inplace=True)
print(f"Удалены колонки: {columns_to_drop}")

here i will delete pointless features in my honest opinion

In [ ]:
to_exclude= {"CNT_FAM_MEMBERS", "FLAG_DOCUMENT_2", "FLAG_DOCUMENT_3", "FLAG_DOCUMENT_4",
                    "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_6", "FLAG_DOCUMENT_7", "FLAG_DOCUMENT_8", "FLAG_DOCUMENT_9",
                    "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11", "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14",
                    "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16", "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19",
                    "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21", "REG_REGION_NOT_LIVE_REGION", "REGION_POPULATION_RELATIVE",
                    "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION",
                    "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "FLAG_PHONE",
                    "FLAG_EMAIL", "WEEKDAY_APPR_PROCESS_START", "ORGANIZATION_TYPE"}

for column in to_exclude:
    df_train = df_train.drop(columns=column)
    df_test = df_test.drop(columns=column)

df_train.sample(15)

next i will divide categorical and numerical columns

In [ ]:
categorical_columns = {"TARGET", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "FLAG_MOBIL", "FLAG_EMP_PHONE",
                        "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY",
                        "NAME_CONTRACT_TYPE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS",
                        "NAME_HOUSING_TYPE"}

numerical_columns = [col for col in df_train.columns if col not in categorical_columns]


def print_unique():
    for feature in categorical_columns:
        if feature in df_train.columns:
            unique_values = df_train[feature].unique()
            print(f"Unique values for {feature}: {unique_values}")
        else:
            print(f"Feature {feature} not found in the DataFrame.")


print_unique()

lets transform categorical features into numerical formats suitable for machine learning by mapping binary values (e.g., "Y"/"N" to 1/0) and ordinal values (e.g., education levels) to a defined priority scale. Additionally, it applies one-hot encoding to multi-class categorical variables to create new binary columns for each unique category.

In [ ]:
df_train["FLAG_OWN_CAR"] = df_train["FLAG_OWN_CAR"].map({"N": 0, "Y": 1})
df_train["FLAG_OWN_REALTY"] = df_train["FLAG_OWN_REALTY"].map({"N": 0, "Y": 1})
df_train["NAME_CONTRACT_TYPE"] = df_train["NAME_CONTRACT_TYPE"].map({"Cash loans": 1, "Revolving loans": 0})
df_test["FLAG_OWN_CAR"] = df_test["FLAG_OWN_CAR"].map({"N": 0, "Y": 1})
df_test["FLAG_OWN_REALTY"] = df_test["FLAG_OWN_REALTY"].map({"N": 0, "Y": 1})
df_test["NAME_CONTRACT_TYPE"] = df_test["NAME_CONTRACT_TYPE"].map({"Cash loans": 1, "Revolving loans": 0})

education_priority = {
    "Lower secondary": 0,
    "Secondary / secondary special": 1,
    "Incomplete higher": 2,
    "Higher education": 3,
    "Academic degree": 4
}
df_train["NAME_EDUCATION_TYPE"] = df_train["NAME_EDUCATION_TYPE"].map(education_priority)
df_test["NAME_EDUCATION_TYPE"] = df_test["NAME_EDUCATION_TYPE"].map(education_priority)

df_train = pd.get_dummies(df_train, columns=[
    "NAME_HOUSING_TYPE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS"
])
df_test = pd.get_dummies(df_test, columns=[
    "NAME_HOUSING_TYPE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS"
])

categorical_int_columns = df_train.select_dtypes(include=['bool']).columns

These functions provide visualization tools for exploring numerical data: histograms display the distribution of features, boxplots reveal outliers and spread, and a heatmap shows correlations among variables in a dataset.

In [ ]:
def plot_numerical_features_histogram(data, numerical_features, bins=30, figsize=(16, 20), color='g', xlabelsize=8,
                                      ylabelsize=8, title="Histograms of Numerical Features"):
    fig = data[numerical_features].hist(
        figsize=figsize,
        color=color,
        bins=bins,
        xlabelsize=xlabelsize,
        ylabelsize=ylabelsize
    )

    plt.suptitle(title, fontsize=16, y=1.02)
    plt.tight_layout()

    plt.show()


def plot_boxplots(data, numerical_features, cols=2, figsize=(10, 4)):
    num_features = len(numerical_features)
    rows = math.ceil(num_features / cols)

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(figsize[0] * cols, figsize[1] * rows))
    fig.tight_layout(pad=3.0)

    axes = axes.flatten() if num_features > 1 else [axes]

    for i, col in enumerate(numerical_features):
        sns.boxplot(x=data[col], ax=axes[i], color='skyblue')
        axes[i].set_title(f'Boxplot for {col}', fontsize=10)
        axes[i].set_xlabel(col, fontsize=8)
        axes[i].set_ylabel('Value', fontsize=8)

    for j in range(len(numerical_features), len(axes)):
        axes[j].axis('off')

    plt.suptitle("Boxplots of Numerical Features", fontsize=16, y=1.02)
    plt.show()


def plot_correlation_heatmap(data, figsize=(16, 12), cmap="Greens", vmax=0.8, annot=True):
    corr_matrix = data.corr()

    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    fig, ax = plt.subplots(figsize=figsize)
    fig.set_size_inches(20, 15)
    sns.heatmap(
        corr_matrix,
        mask=mask,
        cmap=cmap,
        vmax=vmax,
        square=True,
        annot=annot,
        fmt=".2f",
        linewidths=0.5,
        cbar_kws={"shrink": 0.8},
        ax=ax
    )

    plt.title("Correlation Heatmap", fontsize=18, pad=20)
    plt.tight_layout()
    plt.show()

lets get histogram

In [ ]:
plot_numerical_features_histogram(df_train, numerical_columns, bins=20, color='skyblue',
                                  title="Numerical Feature Distributions")

lets extract and lists the names of all numerical columns

In [ ]:
numerical_features = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numerical_features)

correlation heatmap:

In [ ]:
plot_correlation_heatmap(df_train[numerical_features])

then i will exclude columns which correlate the most

In [ ]:
to_exclude = {"AMT_ANNUITY", "AMT_GOODS_PRICE", "OBS_60_CNT_SOCIAL_CIRCLE", "DEF_60_CNT_SOCIAL_CIRCLE",
              "NAME_INCOME_TYPE_Pensioner"}
df_train = df_train.drop(columns=to_exclude)
df_test = df_test.drop(columns=to_exclude)

lets see updated list of numerical features 

In [ ]:
numerical_features = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numerical_features)

here we clean time-related features, compute a social circle median, drop unused columns, and plots numerical features with boxplots.

In [ ]:
#OUTLIERS

df_train['DAYS_EMPLOYED'] = df_train['DAYS_EMPLOYED'].replace(365243, 0).abs()
df_train['DAYS_BIRTH'] = df_train['DAYS_BIRTH'].abs()
df_train['DAYS_REGISTRATION'] = df_train['DAYS_REGISTRATION'].abs()
df_train['DAYS_ID_PUBLISH'] = df_train['DAYS_ID_PUBLISH'].abs()
df_test['DAYS_EMPLOYED'] = df_test['DAYS_EMPLOYED'].replace(365243, 0).abs()
df_test['DAYS_BIRTH'] = df_test['DAYS_BIRTH'].abs()
df_test['DAYS_REGISTRATION'] = df_test['DAYS_REGISTRATION'].abs()
df_test['DAYS_ID_PUBLISH'] = df_test['DAYS_ID_PUBLISH'].abs()

df_test['SOCIAL_CIRCLE_MEDIAN'] = df_test[['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE']].median(axis=1)
df_train['SOCIAL_CIRCLE_MEDIAN'] = df_train[['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE']].median(axis=1)
df_train.drop(['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE'], axis=1, inplace=True)
df_test.drop(['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE'], axis=1, inplace=True)
numerical_features = {"CNT_CHILDREN", "AMT_INCOME_TOTAL", "DAYS_BIRTH", "AMT_CREDIT", "DAYS_EMPLOYED",
                      "DAYS_REGISTRATION", "EXT_SOURCE_2", "DAYS_ID_PUBLISH"}
plot_boxplots(df_train, numerical_features)

Here we prepare the data by splitting it into features and target, handle missing values, apply preprocessing for numerical and categorical columns, build a model pipeline with logistic regression, train the model, and evaluate its performance using ROC AUC.

In [ ]:
from sklearn.impute import SimpleImputer

target_column = 'TARGET'
X = df_train.drop(columns=[target_column])
y = df_train[target_column]

numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

X[numerical_columns] = X[numerical_columns].fillna(X[numerical_columns].median())
X[categorical_columns] = X[categorical_columns].fillna(X[categorical_columns].mode().iloc[0])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')), 
            ('scaler', StandardScaler())
        ]), numerical_columns),

        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
        ]), categorical_columns)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=200, solver='lbfgs', random_state=42))
])

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)
model.fit(train_x, train_y)

test_preds_proba = model.predict_proba(test_x)[:, 1]
roc_auc = roc_auc_score(test_y, test_preds_proba)

fpr, tpr, _ = roc_curve(test_y, test_preds_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Random guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

print(f"ROC AUC Score: {roc_auc:.4f}")

lets get result file

In [ ]:
scaled_test_x = model.named_steps['preprocessor'].transform(df_test)
predicted_probabilities = model.named_steps['classifier'].predict_proba(scaled_test_x)[:, 1]

submission_df = pd.DataFrame({
    "SK_ID_CURR": df_test['SK_ID_CURR'],
    "TARGET": predicted_probabilities
}).to_csv

result: https://imgur.com/a/ISljz9s